In [13]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series, Timestamp
import tensorflow as tf
import tensorflow.compat.v1 as tf1
from tensorflow.compat.v1 import Session, ConfigProto
from tensorflow.python.eager.context import PhysicalDevice
from typing import Dict, List, Union, Generator
import os
from numpy import load
import time

In [14]:
import sys
sys.path.insert(0, '..')

In [15]:
from data_formatters.base import GenericDataFormatter, InputTypes, DataTypes
from data_formatters.sorgenia_wind import SorgeniaFormatter

In [16]:
from expt_settings.configs import ExperimentConfig
from libs.hyperparam_opt import HyperparamOptManager
from libs.tft_model import TemporalFusionTransformer, InterpretableMultiHeadAttention
import libs.utils as utils

# LOAD DATA AND EXTRACT TESTSET

In [5]:
file_path: str = r'C:\Users\Lorenzo\PycharmProjects\TFT\outputs\data\sorgenia_wind\data\sorgenia_wind\data\sorgenia_wind_mm_cop.csv'

In [6]:
raw_data: DataFrame = pd.read_csv(file_path)

In [7]:
raw_data.head()

,plant_name_up,time,kwh,dew_point_2m_C,temperature_2m_C,msl_pressure_hPa,sfc_pressure_hPa,precipitation_1h_mm,wind_speed_mean_10m_1h_ms,wind_speed_mean_100m_1h_ms,...,days_from_start,id,hour,day,day_of_week,month,categorical_id,hours_from_start,categorical_day_of_week,categorical_hour
0,UP_MPNTLCDMRN_1,2016-12-31 23:00:00,225.000,-0.902831,4.980909,1028.935552,983.718765,0.000000e+00,1.381208,2.254668,...,0,UP_MPNTLCDMRN_1,23,31,5,12,UP_MPNTLCDMRN_1,0.0,5,23
1,UP_MPNTLCDMRN_1,2017-01-01 00:00:00,65.625,-0.783022,4.890773,1028.693079,983.524042,-1.734723e-15,1.278206,2.224647,...,0,UP_MPNTLCDMRN_1,0,1,6,1,UP_MPNTLCDMRN_1,1.0,6,0
2,UP_MPNTLCDMRN_1,2017-01-01 01:00:00,31.875,-0.565645,4.807044,1028.156679,983.034144,-1.734723e-15,1.211802,2.130597,...,0,UP_MPNTLCDMRN_1,1,1,6,1,UP_MPNTLCDMRN_1,2.0,6,1
3,UP_MPNTLCDMRN_1,2017-01-01 02:00:00,0.000,-0.331075,4.911959,1028.162779,983.062126,-1.734723e-15,1.017035,1.846413,...,0,UP_MPNTLCDMRN_1,2,1,6,1,UP_MPNTLCDMRN_1,3.0,6,2
4,UP_MPNTLCDMRN_1,2017-01-01 03:00:00,0.000,-0.668040,4.464566,1027.616920,982.551146,-1.734723e-15,0.715094,1.450024,...,0,UP_MPNTLCDMRN_1,3,1,6,1,UP_MPNTLCDMRN_1,4.0,6,3


In [8]:
raw_data['time'] = raw_data['time'].astype('datetime64[s]')

In [17]:
config = ExperimentConfig('sorgenia_wind', r'C:\Users\Lorenzo\PycharmProjects\TFT\outputs')

In [18]:
formatter: SorgeniaFormatter = config.make_data_formatter()

In [19]:
data_csv_path: str = config.data_csv_path

SPLIT DATA

In [12]:
train, valid, test = formatter.split_data(raw_data)

Setting scalers with training data...


In [ ]:
test['time'].unique()[168]

PICK OBSERVED DATA

In [ ]:
forecast_time: str = '2020-08-13 22:00:00'

In [ ]:
obs: DataFrame = test[test['time']<= forecast_time]

In [ ]:
obs

In [ ]:
assert obs.shape[0] == 7*168

PICK KNOWN DATA

In [ ]:
known: DataFrame = test[(test['time'] > forecast_time) & (test['time'] <= pd.Timestamp(forecast_time) + pd.Timedelta(hours=12))]

In [ ]:
known

In [ ]:
assert known.shape[0] == 12*7
assert len(known['time'].unique()) == 12

CONCAT THE TWO DF

In [ ]:
inference_data: DataFrame = pd.concat([obs, known], axis=0, ignore_index=True)
inference_data: DataFrame = inference_data.sort_values(by=['plant_name_up','time'], ascending=True, ignore_index=True)

In [ ]:
inference_data

In [ ]:
assert inference_data.shape[0] == 7*168 + 7*12

In [ ]:
inference_data.columns

PREPARE INFRASTRUCTURE FOR TRAINING

In [20]:
if tf.test.gpu_device_name(): 
    print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

Default GPU Device:/device:GPU:0


In [21]:
gpu: List[PhysicalDevice] = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpu[0], True)

In [22]:
# Tensorflow setup
# default_keras_session: Session = tf1.keras.backend.get_session()
tf_config: ConfigProto = utils.get_default_tensorflow_config(tf_device="gpu", gpu_id=0)

Selecting GPU ID=0


In [23]:
fixed_params: Dict = formatter.get_experiment_params()
fixed_params

{'total_time_steps': 180,
 'num_encoder_steps': 168,
 'num_epochs': 100,
 'early_stopping_patience': 10,
 'multiprocessing_workers': 5,
 'column_definition': [('id', <DataTypes.REAL_VALUED: 0>, <InputTypes.ID: 4>),
  ('time', <DataTypes.DATE: 2>, <InputTypes.TIME: 5>),
  ('kwh', <DataTypes.REAL_VALUED: 0>, <InputTypes.TARGET: 0>),
  ('hour', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>),
  ('day_of_week', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>),
  ('hours_from_start',
   <DataTypes.REAL_VALUED: 0>,
   <InputTypes.KNOWN_INPUT: 2>),
  ('dew_point_2m_C',
   <DataTypes.REAL_VALUED: 0>,
   <InputTypes.OBSERVED_INPUT: 1>),
  ('temperature_2m_C',
   <DataTypes.REAL_VALUED: 0>,
   <InputTypes.OBSERVED_INPUT: 1>),
  ('msl_pressure_hPa',
   <DataTypes.REAL_VALUED: 0>,
   <InputTypes.OBSERVED_INPUT: 1>),
  ('sfc_pressure_hPa',
   <DataTypes.REAL_VALUED: 0>,
   <InputTypes.OBSERVED_INPUT: 1>),
  ('precipitation_1h_mm',
   <DataTypes.REAL_VALUED: 0>,
   <InputTypes.OBSER

In [ ]:
# # Sets up default params
# fixed_params: Dict = formatter.get_experiment_params()
params: Dict = formatter.get_default_model_params()
params["model_folder"]: str = os.path.join(config.model_folder, "fixed")
model_folder = os.path.join(config.model_folder, "fixed")

In [ ]:
params

In [ ]:
# Sets up hyperparam manager
print("*** Loading hyperparm manager ***")
opt_manager = HyperparamOptManager({k: [params[k]] for k in params},
                                   fixed_params, model_folder)

In [ ]:
# opt_manager.hyperparam_folder = r'C:\Users\Lorenzo\savedmodels_sorgenia_wind_mm+cop'
model_folder: str = opt_manager.hyperparam_folder
model_folder

# PREDICTION

In [ ]:
t0: float = time.perf_counter()
print("*** Computing Preds ***")
tf1.reset_default_graph()
with tf.Graph().as_default(), tf1.Session(config=tf_config) as sess:
    tf1.keras.backend.set_session(sess)
    params: Dict = opt_manager.get_next_parameters()
    print(params)
    params['exp_name'] = 'sorgenia_wind'
    params['data_folder'] = config.data_csv_path
    model = TemporalFusionTransformer(params, use_cudnn=True)
    params.pop('exp_name', None)
    params.pop('data_folder', None)
    # load model
    model.load(model_folder, use_keras_loadings=True)
    
#     output_map: Dict = model.predict(inference_data, return_targets=False)
#     p50_forecast: DataFrame = formatter.format_predictions(output_map.get("p50"))

t1: float = time.perf_counter()
print("Time elapsed ", t1-t0)

In [ ]:
p50_forecast